In [ ]:
import math
from manim import *

config.frame_width = 16
config.frame_height = 9
config.frame_size = (854,480)#(3840,2160)#(2560,1440)#(1920,1080)#
config.frame_rate = 30
config.background_color = DARK_GRAY

colors = [
    '#30f2f2',
    '#ffd23f',
    '#ff6f59',
    '#6290c3',
    '#a0a4b8',
    '#ff9b85',
    '#ffd97d',
    '#aaf683',
    '#60d394',
    '#ee6055',
    '#f2d0a9',
    '#f1e3d3',
    '#99c1b9',
    '#8e7dbe',
    '#00bbf9',
    '#00f5d4',
]

In [ ]:
# %%manim -v WARNING --disable_caching T

class T(Scene):
    def construct(self):
        logo = SVGMobject('pics/logo.svg')
        logo.height = 2
        watermark = Text('PccFree.Space', font="Sans", weight=BOLD)#.next_to(logo, DOWN, buff=0)
        watermark.add_updater(lambda _, dt: None)
        brand = VGroup(logo, watermark)

        self.clear()
        # self.add_sound('sounds/pccfs.MP3')
        self.wait(.4)
        self.play(SpiralIn(logo, run_time=2, rate_func=rate_functions.ease_in_quart), AddTextLetterByLetter(watermark))
        self.play(brand.animate(run_time=.5).arrange(aligned_edge=DOWN))
        for i in range(12):
            self.play(watermark.animate(run_time=1/12).set_color(colors[i%12]))
        watermark.set_color(WHITE)
        self.play(Unwrite(brand, run_time=1.5))
        self.clear()

In [ ]:
%%manim -v WARNING --disable_caching T1

class T1(Scene):
    def construct(self):
        cycle_time=2
        trans_time=.5
        radius = 3.25
        dot_radius =.25
        total_num = 12
        silent=False

        hub = Dot(radius=dot_radius).set_stroke(WHITE,4,1).set_z_index(1)
        hub.add_updater(lambda _, dt: None)
        line = Line(UP*radius, DOWN*radius).set_stroke(WHITE,4,1).set_z_index(0)
        dot = Dot(radius=dot_radius).set_stroke(WHITE,12).set_z_index(2)
        time = ValueTracker(0)

        def gen_lines(visible_num, angle=None):
            lines = []
            for i in range(total_num):
                if i < visible_num:
                    if angle == None:
                        lines.append(
                            line.copy().rotate(PI/visible_num*i)
                        )
                    else:
                        lines.append(
                            line.copy().rotate(angle)
                        )
                else:
                    lines.append(
                        line.copy().set_opacity(0.)
                    )
            return VGroup(*lines)
        
        def gen_dots(lines, visible_num):
            def init_prop(i):
                return (1-math.cos(i/visible_num*PI))/2
            def get_proportion(i):
                return math.sin((i/visible_num-.5)*PI + time.get_value()/cycle_time*TAU)*.5+.5
            def get_prop(i):
                return math.sin(i/visible_num*PI/2)*.5+.5
            dots = []
            for i in range(total_num):
                dots.append(dot.copy().set_fill(colors[i]))
                dots[i]._i = i
                dots[i]._c = colors[i]
                dots[i]._p = get_proportion(i)
            def update_dot(dot):
                dot.move_to(
                    lines[dot._i].point_from_proportion(dot._p)
                )
                if dot._i >= visible_num:
                    dot.set_opacity(0)
                else:
                    dot.set_fill(opacity=1).set_stroke(opacity=.1)
                    next_p = get_proportion(dot._i)
                    if (next_p>=.5 and dot._p<.5) or (next_p<=.5 and dot._p>.5):
                        hub.set_stroke(color=dot._c)
                        if not silent:
                            self.add_sound(f'kalimba_f/{13-dot._i}', -.06, -12)
                    dot._p = next_p
            for i in range(total_num):
                dots[i].add_updater(update_dot)
                dots[i].update(0)
            return VGroup(*dots)
        
        def gen_edges(lines, visible_num):
            return Polygon(*[
                    l.points[i] 
                    for i in [0,-1] 
                    for l in lines[:visible_num]
                ]).set_stroke(WHITE, 10, .8)
        
        self.play(FadeIn(hub, run_time=.5))
        # self.add_sound('sounds/T1_1.MP3',4)

        _lines = gen_lines(1)
        _dots = gen_dots(_lines, 1)
        _edges = gen_edges(_lines, 1)
        for visible_num in range(1,13):
            lines = gen_lines(visible_num)
            dots = gen_dots(lines, visible_num)
            edges = gen_edges(lines, visible_num)
            self.play(
                ReplacementTransform(_edges, edges),
                ReplacementTransform(_lines, lines),
                ReplacementTransform(_dots,  dots),
                run_time=trans_time,
                )
            self.play(time.animate(run_time=cycle_time, rate_func=linear).set_value(cycle_time*visible_num))
            _lines=lines
            _dots=dots
            _edges=edges

        time.set_value(0)
        lines = gen_lines(12, angle=0).arrange(buff=1)
        dots = gen_dots(lines, 12)
        silent=True
        self.play(
            FadeOut(_edges), 
            FadeOut(hub), 
            ReplacementTransform(_lines, lines),
            ReplacementTransform(_dots,  dots),
            time.animate.set_value(2),
            run_time=2,
            rate_func=linear,
            suspend_mobject_updating=False,
            )
        self.play(time.animate(run_time=4, rate_func=linear).set_value(6))

        ax = Axes(
            x_range=[0, 11, 1],
            y_range=[0, 1, .5],
            x_length=11,
            y_length=radius*2,
            tips=False,
            axis_config={"include_numbers": True},
        )
        graph = always_redraw(
            lambda : ax.plot(
                lambda i: math.sin((i/12+.5)*PI + time.get_value()/cycle_time*TAU)*.5+.5
            )
        )
        self.play(FadeIn(graph),
            FadeOut(lines), 
            FadeOut(dots),
            Create(ax), 
        )
        self.play(
            time.animate(run_time=6, rate_func=linear).set_value(12),
        )
        T.construct(self)
#####################################################
        time.set_value(0)
        _lines = gen_lines(12)
        _dots = gen_dots(_lines, 12)
        _edges = gen_edges(_lines, 12)
        lines = gen_lines(12, angle=0).arrange(buff=1)
        dots = gen_dots(lines, 12)
        graph = ax.plot(
            lambda i: math.sin((i/12-.5)*PI + time.get_value()/cycle_time*TAU)*.5+.5
        )
        # self.add_sound('sounds/T1_2.MP3')
        self.play(
            FadeIn(_lines, lag_ratio=1), 
            Create(_edges), 
            run_time=4, 
            rate_func=linear,
        )
        self.add(hub, _dots)
        self.wait(6)
        self.play(Flash(_dots[0], flash_radius=dot_radius, color=colors[0]))
        self.wait()
        self.play(Indicate(_dots[6], color=colors[6], scale_factor=1.5))
        self.wait(2)
        self.play(
            FadeOut(hub), 
            FadeOut(_edges), 
            ReplacementTransform(_lines, lines),
            ReplacementTransform(_dots,  dots),
            run_time=3,
            suspend_mobject_updating=False,
        )
        self.wait()
        self.play(lines.animate.flip(RIGHT), dots.animate.flip(RIGHT), run_time=1)
        self.wait(2)
        self.play(Create(graph))
        self.wait()
        self.remove(dots)
        self.play(Unwrite(lines), Create(ax, run_time=2))
        _ax = ax
        _graph = graph
        ax = Axes(
            x_range=[0, 11, 1],
            y_range=[-1, 1, 1],
            x_length=10,
            y_length=radius*2,
            tips=False,
            axis_config={"include_numbers": True},
        ).shift(LEFT*2)
        _sin_graph = _ax.plot(lambda x: math.sin(x))
        sin_graph_0 = ax.plot(lambda x: math.sin(x))
        sin_label_0 = ax.get_graph_label(sin_graph_0, 'sin(i)')
        sin_graph_1 = ax.plot(
            lambda i: math.sin(i/12)
        )
        sin_label_1 = ax.get_graph_label(sin_graph_1, 'sin(\\frac{i}{n})')
        sin_graph_2 = ax.plot(
            lambda i: math.sin(i/12*PI)
        )
        sin_label_2 = ax.get_graph_label(sin_graph_2, 'sin(\\pi(\\frac{i}{n}))')
        sin_graph_3 = ax.plot(
            lambda i: math.sin((i/12-.5)*PI)
        )
        sin_label_3 = ax.get_graph_label(sin_graph_3, 'sin(\\pi(\\frac{i}{n}-\\frac{1}{2}))')
        sin_graph_4 = ax.plot(
            lambda i: math.sin((i/12-.5)*PI)*.5
        )
        sin_label_4 = ax.get_graph_label(sin_graph_4, '\\frac{1}{2}sin(\\pi(\\frac{i}{n}-\\frac{1}{2}))')
        sin_graph_5 = ax.plot(
            lambda i: math.sin((i/12-.5)*PI)*.5+.5
        )
        sin_label_5 = ax.get_graph_label(sin_graph_5, '\\frac{1}{2}sin(\\pi(\\frac{i}{n}-\\frac{1}{2}))+\\frac{1}{2}')
        graph = ax.plot(
            lambda i: math.sin((i/12-.5)*PI + time.get_value()/cycle_time*TAU)*.5+.5
        )
        self.wait()
        self.play(Create(_sin_graph, run_time=2))
        self.play(
            ReplacementTransform(_sin_graph, sin_graph_0),
            ReplacementTransform(_graph, graph),
            ReplacementTransform(_ax, ax),
            run_time=2,
        )
        self.play(Write(sin_label_0))
        self.wait()
        for i in range(5):
            self.play(
                ReplacementTransform(
                    locals()[f'sin_graph_{i}'], locals()[f'sin_graph_{i+1}']
                ),
                ReplacementTransform(
                    locals()[f'sin_label_{i}'], locals()[f'sin_label_{i+1}']
                ),
            )
            self.wait()
        self.play(
            Unwrite(VGroup(ax,sin_graph_5,graph)),
            sin_label_5.animate.center(),
            run_time=2,
        )
        self.wait()
        self.clear()

        silent = False
        lines = gen_lines(1, PI/2)
        dots = gen_dots(lines, 1)
        self.play(FadeIn(VGroup(lines, dots)))
        self.play(time.animate(run_time=4, rate_func=linear).set_value(4))
        label = MathTex(
            '\\frac{1}{2}sin(\\pi(\\frac{i}{n}-\\frac{1}{2})+\\frac{time}{cycle\\ time}\\times 2\\pi)+\\frac{1}{2}'
            ).next_to(sin_label_5,DOWN, aligned_edge=LEFT)
        self.play(
            FadeOut(VGroup(lines, dots)),
            FadeIn(sin_label_5),
        )
        self.wait()
        self.play(Write(label))
        self.wait()
        self.play(
            label.animate.move_to(sin_label_5),
            FadeOut(sin_label_5),
        )

        time.set_value(0)
        ax = Axes(
            x_range=[0, 4, 1],
            y_range=[0, 1, .5],
            x_length=12,
            y_length=radius*2,
            tips=False,
            axis_config={"include_numbers": True},
        )
        graph = ax.plot(
                lambda t: math.sin((0/12-.5)*PI + t/cycle_time*TAU)*.5+.5
            )
        text_time = MathTex('time=').next_to(ax, DOWN, aligned_edge=LEFT)
        text_data = DecimalNumber(0).next_to(text_time)
        text_data.add_updater(lambda m: m.set_value(time.get_value()))

        self.wait()
        self.play(label.animate.next_to(ax,UP,buff=0))
        self.play(Create(ax), Create(graph), run_time=2)
        self.wait(3)

        _lines = gen_lines(1,PI).shift(LEFT*6)
        _dots = gen_dots(_lines, 1)
        self.play(FadeIn(_dots), FadeIn(_lines))
        self.play(
            time.animate.set_value(4),
            _lines.animate.shift(RIGHT*12),
            run_time=4,
            rate_func=linear,
            suspend_mobject_updating=False,
        )
        self.wait(2)
        self.remove(_lines, _dots)

        _lines = gen_lines(12,PI).arrange(buff=.25).shift(LEFT*4.5)
        _dots = gen_dots(_lines, 12)
        self.play(FadeIn(_dots), FadeIn(_lines))
        self.play(
            time.animate.set_value(7),
            _lines.animate.shift(RIGHT*9),
            run_time=3,
            rate_func=linear,
            suspend_mobject_updating=False,
        )
        self.play(FadeOut(label), FadeOut(graph), FadeOut(ax))
        lines = gen_lines(12)
        dots = gen_dots(lines, 12)
        edges = gen_edges(lines, 12)
        time.set_value(1)
        self.play(
            ReplacementTransform(_lines, lines),
            ReplacementTransform(_dots,  dots),
            Write(VGroup(text_time, text_data)),
            run_time=3,
            suspend_mobject_updating=False,
        )
        self.play(time.animate(run_time=5, rate_func=linear).set_value(6))
        self.play(
            time.animate.set_value(10),
            Create(edges),
            run_time=4, 
            rate_func=linear
        )
        self.remove(text_time, text_data)
        _lines=lines
        _dots=dots
        _edges=edges
        lines = gen_lines(4)
        dots = gen_dots(lines, 4)
        edges = gen_edges(lines, 4)
        time.set_value(0)
        self.play(
            ReplacementTransform(_edges, edges),
            ReplacementTransform(_lines, lines),
            ReplacementTransform(_dots,  dots),
            Write(hub),
        )
        self.play(time.animate(run_time=20, rate_func=linear).set_value(20))
        T.construct(self)